In [45]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

<h2>Finding out the mean and std dev of the training dataset</h2>

In [46]:
# dataset= CIFAR10(root="data", train= True, download=True, transform= transforms.ToTensor())
# loader= DataLoader(dataset, batch_size= 50000, shuffle= True)

# data= next(iter(loader))[0]
# mean= data.mean(dim=[0, 2, 3])
# std= data.std(dim=[0, 2, 3])

# mean= tuple(mean.tolist())
# std= tuple(std.tolist())

# print("Mean: ", mean)
# print("Standard Deviation: ", std)
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

print("Mean: ", mean)
print("Standard Deviation: ", std)


Mean:  (0.4914, 0.4822, 0.4465)
Standard Deviation:  (0.2023, 0.1994, 0.201)


In [47]:
#next(iter(loader))[0].shape

<h2>Further Steps</h2>

In [48]:
train_transform= transforms.Compose([
    transforms.RandomCrop(32, padding= 4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),      # This must come after the augmentations
    transforms.Normalize(mean, std)
])

# Added this later after bad test loss and accuracy
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [49]:
# For the splits to apply individul tranforms
base_train = CIFAR10(root="data", train=True, download=True, transform=train_transform)
base_val = CIFAR10(root="data", train=True, download=True, transform=val_transform)

torch.manual_seed(0)

train_set, _ = torch.utils.data.random_split(base_train, [45000, 5000])
_, val_set = torch.utils.data.random_split(base_val, [45000, 5000])


test_set = CIFAR10(root="data", train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = DataLoader(val_set, batch_size=128, shuffle = False)
test_loader = DataLoader(test_set, batch_size=128, shuffle = False)

In [50]:
len(train_set), len(val_set), len(test_set)

(45000, 5000, 10000)

In [51]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
# Input size of images is 32x32

    self.stack= nn.Sequential(
        nn.Conv2d(3, 32, 3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, 3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),


        # Block 2
        nn.Conv2d(32, 64, 3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),


        # Block 3
        nn.Conv2d(64, 128, 3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, 128, 3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),


    )

    self.classifier= nn.Sequential(
        nn.Flatten(),
        nn.Linear(128*4*4, 256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(256, 10) #Final 10 outputs
    )

  def forward(self, x):
    x= self.stack(x)
    x= self.classifier(x)
    return x

In [52]:
model= CNN().to(device)

In [53]:
loss_function= nn.CrossEntropyLoss()

In [54]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [55]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [56]:
def validate(dataloader, model, loss_function):
  model.eval()
  total_loss= 0
  correct= 0
  total= 0
  with torch.no_grad():
    for image, label in dataloader:
      image= image.to(device)
      label= label.to(device)

      pred = model(image)
      loss= loss_function(pred, label)
      total_loss+= loss

      predicted_classes = pred.argmax(dim=1)
      correct+= (predicted_classes == label).sum().item()
      total+= label.size(0)

  avg_loss= total_loss / len(dataloader)
  accuracy= correct/total * 100
  print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

  return accuracy

In [57]:
def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for image, label in dataloader:
            image= image.to(device)
            label= label.to(device)

            preds = model(image)
            loss = loss_function(preds, label)
            total_loss += loss.item()

            predicted_classes = preds.argmax(dim=1)
            correct += (predicted_classes == label).sum().item()
            total += label.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total * 100
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

In [58]:
epochs = 25
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_acc = validate(val_loader, model, loss_function)
    scheduler.step(val_acc)


Epoch 1/25
Training Average Loss: 1.5975
Validation Loss: 1.3060, Validation Accuracy: 52.30%

Epoch 2/25
Training Average Loss: 1.1874
Validation Loss: 0.9947, Validation Accuracy: 63.68%

Epoch 3/25
Training Average Loss: 1.0150
Validation Loss: 0.8151, Validation Accuracy: 70.94%

Epoch 4/25
Training Average Loss: 0.9160
Validation Loss: 0.8482, Validation Accuracy: 70.56%

Epoch 5/25
Training Average Loss: 0.8437
Validation Loss: 0.7386, Validation Accuracy: 73.22%

Epoch 6/25
Training Average Loss: 0.7920
Validation Loss: 0.7456, Validation Accuracy: 74.10%

Epoch 7/25
Training Average Loss: 0.7519
Validation Loss: 0.5985, Validation Accuracy: 79.88%

Epoch 8/25
Training Average Loss: 0.7075
Validation Loss: 0.5740, Validation Accuracy: 80.56%

Epoch 9/25
Training Average Loss: 0.6787
Validation Loss: 0.5863, Validation Accuracy: 79.48%

Epoch 10/25
Training Average Loss: 0.6531
Validation Loss: 0.5133, Validation Accuracy: 82.72%

Epoch 11/25
Training Average Loss: 0.6238
Valida

In [59]:
test(test_loader, model, loss_function)

Test Loss: 0.4732, Test Accuracy: 84.39%


<h1>Learning</h1>

**Key Point:** <br>
The code applies data augmentation transformations—such as random cropping, flipping, rotation, and color jitter—to the entire dataset (`train_data`) before splitting it into training and validation sets. As a result, the validation set also receives these augmentations, which makes validation metrics unrepresentative of actual test set performance.

**Result:**
Since the validation set contains augmented data, it may be “easier” or at least different from the real, un-augmented test data. This can lead to inflated validation accuracy. When the model is then evaluated on the true test set, which lacks such augmentations, performance may drop significantly.

**Solution:**
To address this, augmentations should be applied **only** to the training subset. The validation and test datasets should not be augmented; instead, they should only be normalized. This can be done by defining a separate `val_transform` that mirrors the `test_transform`, using only `ToTensor()` and `Normalize()`.

**<h2>Important</h2>**

 `ToTensor()` must come after the augmentations that operate on PIL Images. `ColorJitter`, `RandomCrop`, `RandomHorizontalFlip`, and `RandomRotation` work on PIL, not Tensors.